In [26]:
from tsai.models import InceptionTime, TST, ResNet
# from tsai.data.core import get_UCR_data
# from tsai.learner import TSClassifier
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from CNN_classifier_model import CNNClassifier, train_model, evaluate_model
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels, create_time_windows_with_metadata
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")
from tsai.models import ResNet
from tsai.models.ResNet import ResNet
from fastai.metrics import accuracy
from fastai.data.core import DataLoaders
from fastai.learner import Learner
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import SGD
from torch.optim import AdamW
from functools import partial
from fastai.optimizer import OptimWrapper


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


/home/nwertheim/miniconda3/bin/python


In [27]:
import wandb
from fastai.callback.wandb import WandbCallback

# Initialize WandB logging
wandb.init(project='ResNet Aggregated Predictions')

In [28]:
# Final Hyperparameters
batch_size = 64
learning_rate = 7.991870883780338e-03
num_epochs = 5

In [29]:

'''MY DATA + class weights'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test_2.npy")
test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import SGD
from torch.optim import AdamW
from functools import partial
from fastai.optimizer import OptimWrapper
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



from collections import Counter

train_windows, train_labels, train_meta = create_time_windows_with_metadata(train_data)
test_windows, test_labels, test_meta = create_time_windows_with_metadata(test_data)


744
192
{'record_name': 'Hopper-2024_11_03_14_13_16-0000010179-0002-chan0', 'signal': array([[ 0.70569938],
       [ 1.58579907],
       [ 1.22047813],
       ...,
       [-0.30476434],
       [-0.16021862],
       [-0.40381152]]), 'fs': 20, 'preterm': 1}


In [30]:
# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
test_labels_tensor = test_labels_tensor.long()


print(train_labels_tensor.shape)

train_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_windows_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Test metadata for reference
test_meta = test_meta


torch.Size([9900])


In [32]:
from tsai.models import ResNet
from tsai.models.ResNet import ResNet
from fastai.metrics import accuracy
from fastai.data.core import DataLoaders
from fastai.learner import Learner
import torch

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dls = DataLoaders(train_loader, test_loader)

# Define model
model = ResNet(1, 2)

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Use weighted loss
# loss_func = CrossEntropyLossFlat(weight=class_weights_tensor) # CrossEntropyLossFlat
loss_func=torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
# Replace your loss function in the learner
# SGD OPITMIZER
learn = Learner(
    dls, 
    model, 
    loss_func=loss_func, 
    metrics=accuracy, 
    opt_func=SGD,  # Use Stochastic Gradient Descent
    cbs=[WandbCallback()]  # Add WandbCallback here to log training data to W&B
)

# Optionally convert model to FP32 (if it was FP16 before), and ensure model is on correct device
learn.model.to(device)

# learn.dls.to(device)
# Train
learn.fit_one_cycle(5, learning_rate)

Could not gather input dimensions
WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'TensorDataset' object has no attribute 'items'


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [ ]:
preds, targs = learn.get_preds(dl=test_loader)
# Convert predictions and true labels to numpy arrays
preds = preds.numpy().flatten()
targs = targs.numpy().flatten()

# Assuming test_meta contains a list of identifiers (e.g., window IDs)
pred_df = pd.DataFrame({
    'window_id': [item for sublist in test_meta for item in sublist],  # Flatten the metadata if needed
    'prediction': preds,
    'target': targs
})

# Save predictions with metadata to a CSV file
pred_df.to_csv('predictions_with_metadata.csv', index=False)


In [10]:
# from torch.utils.data import Dataset

# class SignalWindowDataset(Dataset):
#     def __init__(self, windows, labels, metadata):
#         self.windows = torch.tensor(windows, dtype=torch.float32).unsqueeze(1)  # shape (N, 1, T)
#         self.labels = torch.tensor(labels, dtype=torch.long)
#         self.metadata = metadata

#     def __len__(self):
#         return len(self.windows)

#     def __getitem__(self, idx):
#         return self.windows[idx], self.labels[idx], self.metadata[idx]

# train_dataset = SignalWindowDataset(train_windows, train_labels, train_meta)
# test_dataset = SignalWindowDataset(test_windows, test_labels, test_meta)

# # Then use standard PyTorch DataLoaders (not `fastai`'s DataLoaders)
# from torch.utils.data import DataLoader

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
# from fastai.data.core import DataLoaders

# dls = DataLoaders(train_loader, test_loader)

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [14]:
# from fastai.callback.core import Callback

# class SavePredsWithMetaCallback(Callback):
#     def __init__(self, test_meta, output_path):
#         self.test_meta = test_meta  # Your metadata (e.g., file names or window identifiers)
#         self.output_path = output_path
#         self.preds = []
#         self.targets = []
#         self.ids = []  # This will store window identifiers

#     def before_batch(self):
#         self.batch_preds = []  # Reset before each batch

#     def after_pred(self):
#         # Store predictions and corresponding metadata
#         self.preds.append(self.pred)
#         self.targets.append(self.yb)
#         self.ids.append(self.test_meta)  # Append metadata (window identifier)

#     def after_fit(self):
#         # After the training finishes, save the predictions and metadata
#         pred_df = pd.DataFrame({
#             'window_id': [item for sublist in self.ids for item in sublist],
#             'prediction': [item for sublist in self.preds for item in sublist],
#             'target': [item for sublist in self.targets for item in sublist]
#         })
#         pred_df.to_csv(self.output_path, index=False)


In [15]:

# model = ResNet(1, 2).to(device)
# loss_func=torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# learn = Learner(
#     dls,
#     model,
#     loss_func=loss_func,
#     metrics=accuracy,
#     opt_func=SGD,
#     cbs=[SavePredsWithMetaCallback(test_meta=test_meta, output_path="results/preds_with_meta.csv")]
# )

# learn.fit_one_cycle(5, learning_rate)




epoch,train_loss,valid_loss,accuracy,time


TypeError: ResNet.forward() takes 2 positional arguments but 3 were given